In [1]:
import os
from IPython.display import display

# ROOT contais the project directory 
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from src.plots import plot_color_bar
from src.Fundus import Fundus
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from tqdm import tqdm
import torch

In [3]:
def cluster_pixels(fundus, n):
    pal = fundus.palette
    
    cluster = AgglomerativeClustering(n_clusters=n, affinity='euclidean', linkage='ward')
    clustered = cluster.fit_predict(pal)

    a = np.zeros(fundus.pixels.shape, dtype=np.uint8)
    for c in tqdm(np.unique(clustered)):
        a+=fundus.mask(pal[clustered == c], inverse=True, replacement=pal[clustered == c][0])
        
    return Fundus(a, w=fundus.w, h=fundus.h)

In [35]:
def nearest_neighbour_mean(x):

    def cluster(x):
        out = []
        # Iterate each row
        for i in range(len(x)):
            if i-1 < 0:
                out.append(torch.mean(x[i:i+2], dim=0))

            elif i+1 < len(x):
                frame = torch.stack([x[i-1], x[i+1]]).to("cuda")
                diffs = torch.stack([ abs(x[i]-x[i-1]), abs(x[i]-x[i+1])])
                index = torch.min(diffs, dim=0).indices[:,0]
                minvl = torch.stack([frame[j,i,:] for i,j in enumerate(index)])
                out.append(torch.mean(torch.stack([minvl, x[i]]), dim=0))

            else:
                out.append(torch.mean(x[i-1:i+1], dim=0))
        # Stack 
        out = torch.stack(out, dim=0)
        return out.type(torch.int16)
    
    # Run clustering on data
    meaned_v = cluster(x)
    meaned_h = cluster(torch.transpose(x, 0, 1))
    
    # Differrence between original and meaned
    diff_v = x - meaned_v
    diff_h = x - meaned_h
    
    # Get clossest (nearest neighbour)
    nnm = torch.where(abs(diff_h[:,:,0]) < abs(diff_v[:,:,0]), meaned_v, meaned_h)
    
    return nnm

In [36]:
original = Fundus("data/noma/male_extra_0.png")
len(original.palette)

34070

In [37]:
originaltensor = torch.tensor(np.asarray(original.im)).to("cuda")

In [38]:
nnm = nearest_neighbour_mean(originaltensor.type(torch.float16))

RuntimeError: The size of tensor a (2048) must match the size of tensor b (1536) at non-singleton dimension 1

In [ ]:
modified = cluster_pixels(original, len(original.palette)//2)
modified2 = cluster_pixels(modified, len(modified.palette)//2)
modified3 = cluster_pixels(modified2, len(modified2.palette)//2)

In [ ]:
veinsubset =  Fundus(modified2.im.crop((550, 550, 600, 600)))
veinsubset.im

In [ ]:
#colors = sorted(veinsubset.palette, key=lambda x: x[1])
colors = np.sort(veinsubset.palette, axis=0)[:120]
plot_color_bar(colors)

In [ ]:
veinsubset.mask(colors, inplace=False)
Fundus(modveinsubset, w=veinsubset.w, h=veinsubset.h).im

In [ ]:
simp_vein = modified2.mask(colors=colors, inplace=False)
Fundus(simp_vein, w=original.w, h=original.h).im

In [ ]:
def nearest_neighbour_mean(x):

    def cluster(x):
        out = []
        # Iterate each row
        for i in range(len(x)):
            if i-1 < 0:
                out.append(torch.mean(x[i:i+2], dim=0))

            elif i+1 < len(x):
                frame = torch.stack([x[i-1], x[i+1]]).to("cuda")
                diffs = torch.stack([ abs(x[i]-x[i-1]), abs(x[i]-x[i+1])])
                index = torch.min(diffs, dim=0).indices[:,0]
                minvl = torch.stack([frame[j,i,:] for i,j in enumerate(index)])
                out.append(torch.mean(torch.stack([minvl, x[i]]), dim=0))

            else:
                out.append(torch.mean(x[i-1:i+1], dim=0))
        # Stack 
        out = torch.stack(out, dim=0)
        return out.type(torch.int16)
    
    # Run clustering on data
    meaned_v = cluster(x)
    meaned_h = cluster(torch.transpose(x, 0, 1))
    
    # Differrence between original and meaned
    diff_v = x - meaned_v
    diff_h = x - meaned_h
    
    # Get clossest (nearest neighbour)
    nnm = torch.where(abs(diff_h[:,:,0]) < abs(diff_v[:,:,0]), meaned_v, meaned_h)
    
    return nnm

In [ ]:
arr = np.asarray(original.im)
arr = torch.tensor(arr).to("cuda").type(torch.float32)

In [ ]:
nnm = nearest_neighbour_mean(arr)

In [ ]:
mod = Image.fromarray(nnm.to("cpu").numpy().astype(np.uint8))
mod = Fundus(mod)

In [ ]:
len(mod._get_palette())

In [ ]:
mod.im